In [ ]:
# ================================
# Standard library imports
# ================================
import json
# ================================
# Third-party imports
# ================================
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display, HTML
# ================================
# Local / project imports
# ================================
import research_tools
# ================================
# Environment setup
# ================================
load_dotenv()  # Load environment variables from .env file

# Instantiate OpenAI's client (you should use this in your graded functions)
CLIENT = OpenAI()

### Tools to find research papers using ArXiv and Tavily

In [6]:
# Test the arXiv search tool
topic = "Agentic AI"

arxiv_results = research_tools.arxiv_search_tool(topic, max_results=3)

# Show formatted arxiv_results
for i, paper in enumerate(arxiv_results, 1):
    if "error" in paper:
        print(f"❌ Error: {paper['error']}")
    else:
        print(f"📄 Paper {i}")
        print(f"  Title     : {paper['title']}")
        print(f"  Authors   : {', '.join(paper['authors'])}")
        print(f"  Published : {paper['published']}")
        print(f"  URL       : {paper['url']}\n")


print("\n🧾 Raw arxiv_Results:\n")
print(json.dumps(arxiv_results, indent=2))

📄 Paper 1
  Title     : AI Agents and Agentic AI-Navigating a Plethora of Concepts for Future
  Manufacturing
  Authors   : Yinwang Ren, Yangyang Liu, Tang Ji, Xun Xu
  Published : 2025-07-02
  URL       : http://arxiv.org/abs/2507.01376v1

📄 Paper 2
  Title     : Responsible AI Agents
  Authors   : Deven R. Desai, Mark O. Riedl
  Published : 2025-02-25
  URL       : http://arxiv.org/abs/2502.18359v1

📄 Paper 3
  Title     : Agentic AI and Multiagentic: Are We Reinventing the Wheel?
  Authors   : V. Botti
  Published : 2025-06-02
  URL       : http://arxiv.org/abs/2506.01463v1


🧾 Raw arxiv_Results:

[
  {
    "title": "AI Agents and Agentic AI-Navigating a Plethora of Concepts for Future\n  Manufacturing",
    "authors": [
      "Yinwang Ren",
      "Yangyang Liu",
      "Tang Ji",
      "Xun Xu"
    ],
    "published": "2025-07-02",
    "url": "http://arxiv.org/abs/2507.01376v1",
    "summary": "AI agents are autonomous systems designed to perceive, reason, and act within\ndynamic en

In [ ]:
# Test the tavily search tool
topic = "Agentic AI"

tavily_results = research_tools.tavily_search_tool(topic, max_results=3, include_images=False)

# Show formatted arxiv_results
for i, paper in enumerate(tavily_results, 1):
    if "error" in paper:
        print(f"❌ Error: {paper['error']}")
    else:
        print(f"📄 Paper {i}")
        print(f"  Title     : {paper['title']}")
        print(f"  Content   : {paper['content']}")
        print(f"  URL       : {paper['url']}\n")


print("\n🧾 Raw tavily_Results:\n")
print(json.dumps(tavily_results, indent=2))

📄 Paper 1
  Title     : Agentic AI - Wikipedia
  Content   : **Agentic AI** is a class of artificial intelligence that focuses on autonomous systems that can make decisions and perform tasks with limited or no human intervention. While robotic process automation (RPA) systems automate rule-based, repetitive tasks with fixed logic, agentic AI adapts based on data inputs. Agentic AI refers to autonomous systems capable of pursuing complex goals with minimal human intervention, often making decisions based on continuous learning and external data. | Applications | * Machine learning   + In-context learning * Artificial neural network "Neural network (machine learning)")   + Deep learning * Language model   + Large   + NMT   + Reasoning * Model Context Protocol * Intelligent agent * Artificial human companion * Humanity's Last Exam * Artificial general intelligence (AGI) |
  URL       : https://en.wikipedia.org/wiki/Agentic_AI

📄 Paper 2
  Title     : What is Agentic AI? - AWS
  Content   

In [12]:
# Tool mapping
TOOL_MAPPING = {
    "tavily_search_tool": research_tools.tavily_search_tool,
    "arxiv_search_tool": research_tools.arxiv_search_tool,
}

### Step 1 - Generate a Research Report with the tools

In [13]:
# generate_research_report_with_tools
def generate_research_report_with_tools(prompt: str, model: str = "gpt-4o") -> str:
    """
    Generates a research report using OpenAI's tool-calling with arXiv and Tavily tools.

    Args:
        prompt (str): The user prompt.
        model (str): OpenAI model name.

    Returns:
        str: Final assistant research report text.
    """
    messages = [
        {
            "role": "system",
            "content": (
                "You are a research assistant that can search the web and arXiv to write detailed, "
                "accurate, and properly sourced research reports.\n\n"
                "🔍 Use tools when appropriate (e.g., to find scientific papers or web content).\n"
                "📚 Cite sources whenever relevant. Do NOT omit citations for brevity.\n"
                "🌐 When possible, include full URLs (arXiv links, web sources, etc.).\n"
                "✍️ Use an academic tone, organize output into clearly labeled sections, and include "
                "inline citations or footnotes as needed.\n"
                "🚫 Do not include placeholder text such as '(citation needed)' or '(citations omitted)'."
            )
        },
        {"role": "user", "content": prompt}
    ]

    # List of available tools
    tools = [research_tools.arxiv_tool_def, research_tools.tavily_tool_def]

    # Maximum number of turns
    max_turns = 3
    
    # Iterate for max_turns iterations
    for _ in range(max_turns):

        ### START CODE HERE ###

        # Chat with the LLM via the client and set the correct arguments. Hint: Their names match names of variables already defined.
        # Make sure to let the LLM choose tools automatically. Hint: Look at the docs provided earlier!
        response = CLIENT.chat.completions.create( 
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto",
            temperature=1, 
        ) 

        ### END CODE HERE ###

        # Get the response from the LLM and append to messages
        msg = response.choices[0].message 
        messages.append(msg) 

        # Stop when the assistant returns a final answer (no tool calls)
        if not msg.tool_calls:      
            final_text = msg.content
            print("✅ Final answer:")
            print(final_text)
            break

        # Execute tool calls and append results
        for call in msg.tool_calls:
            tool_name = call.function.name
            args = json.loads(call.function.arguments)
            print(f"🛠️ {tool_name}({args})")

            try:
                tool_func = TOOL_MAPPING[tool_name]
                result = tool_func(**args)
            except Exception as e:
                result = {"error": str(e)}

            ### START CODE HERE ###

            # Keep track of tool use in a new message
            new_msg = { 
                # Set role to "tool" (plain string) to signal a tool was used
                "role": "tool",
                # As stated in the markdown when inspecting the ChatCompletionMessage object 
                # every call has an attribute called id
                "tool_call_id": call.id,
                # The name of the tool was already defined above, use that variable
                "name": tool_name,
                # Pass the result of calling the tool to json.dumps
                "content": json.dumps(None)
            }

            ### END CODE HERE ###

            # Append to messages
            messages.append(new_msg)

    return final_text

In [14]:
report = generate_research_report_with_tools("Agentic AI")

print(report)

🛠️ arxiv_search_tool({'query': 'Agentic AI', 'max_results': 5})
🛠️ tavily_search_tool({'query': 'Agentic AI', 'max_results': 5})
✅ Final answer:
It appears there are no direct results available from the search tools regarding "Agentic AI." However, I can provide a general overview and potential implications of Agentic AI based on existing knowledge.

### Overview of Agentic AI

Agentic AI refers to artificial intelligence systems that are designed to exhibit agency, meaning they can perceive their environment and take actions autonomously to achieve specific goals. This concept extends beyond traditional AI, which often operates under fixed rules and lacks the capability to independently devise strategies or make decisions that reflect self-initiative or adaptive behavior. 

#### Key Characteristics:
1. **Autonomy**: Agentic AI systems can operate independently without human intervention, making decisions based on their programming and dynamic inputs from their environment.
2. **Goal-D

### Step 2 - Reflect and Refine the report

In [15]:
# reflection_and_rewrite
def reflection_and_rewrite(report, model: str = "gpt-4o-mini", temperature: float = 0.3) -> dict:
    """
    Generates a structured reflection AND a revised research report.
    Accepts raw text OR the messages list returned by generate_research_report_with_tools.

    Returns:
        dict with keys:
          - "reflection": structured reflection text
          - "revised_report": improved version of the input report
    """

    # Input can be plain text or a list of messages, this function detects and parses accordingly
    report = research_tools.parse_input(report)

    ### START CODE HERE ###

    # Define the prompt. A multi-line f-string is typically used for this.
    # Remember it should ask the model to output ONLY valid JSON with this structure:
    # {{ "reflection": "<text>", "revised_report": "<text>" }}
    user_prompt = f'''
    
    Using the report from: {report}
    
    Return a dict in the following format containing 
    the two deliverables with the following keys:
    
    {{ "reflection": "<text>", "revised_report": "<text>" }}
    
    
    The reflection should cover :
    - strengths
    - limitations
    - suggestions
    - opportunities
    
    The revised report should incorporate these elements to improve clarity and academic tone.
    
    
    
    
    '''

    # Get a response from the LLM
    response = CLIENT.chat.completions.create( 
        # Pass in the model
        model="gpt-4o-mini",
        messages=[ 
            # System prompt is already defined
            {"role": "system", "content": "You are an academic reviewer and editor."},
            # Add user prompt
            {"role": "user", "content": user_prompt},
        ],
        # Set the temperature equal to the temperature parameter passed to the function
        temperature=0.3
    )

    ### END CODE HERE ###

    # Extract output
    llm_output = response.choices[0].message.content.strip()

    # Check if output is valid JSON
    try:
        data = json.loads(llm_output)
    except json.JSONDecodeError:
        raise Exception("The output of the LLM was not valid JSON. Adjust your prompt.")

    return {
        "reflection": str(data.get("reflection", "")).strip(),
        "revised_report": str(data.get("revised_report", "")).strip(),
    }

In [16]:
new_report = reflection_and_rewrite(report)

print(new_report)

{'reflection': '<text>Strengths: The report provides a clear and concise overview of Agentic AI, effectively outlining its key characteristics and implications. The structure is logical, making it easy for readers to follow the discussion. The inclusion of various fields impacted by Agentic AI demonstrates a comprehensive understanding of its potential reach. \n\nLimitations: While the report touches on important ethical and governance issues, it lacks depth in discussing specific frameworks or examples of how these concerns might be addressed. Additionally, the economic impact section could benefit from more detailed analysis or case studies to illustrate potential disruptions in labor markets. \n\nSuggestions: To enhance the report, consider incorporating specific examples of existing agentic AI systems and their applications. Including a discussion on the current state of research and development in this area could also provide valuable context. Furthermore, expanding on the ethical

### Step 3 - Convert the Revised report to HTML

In [18]:
# GRADED FUNCTION: convert_report_to_html
def convert_report_to_html(report, model: str = "gpt-4o", temperature: float = 0.5) -> str:
    """
    Converts a plaintext research report into a styled HTML page using OpenAI.
    Accepts raw text OR the messages list from the tool-calling step.
    """

    # Input can be plain text or a list of messages, this function detects and parses accordingly
    report = research_tools.parse_input(report)

    # System prompt is already provided
    system_prompt = "You convert plaintext reports into full clean HTML documents."

    ### START CODE HERE ###
    
    # Build the user prompt instructing the model to return ONLY valid HTML
    user_prompt = f'''
    
    You have received a report in plaintext: {report}
    
    You need convert into a clean HTML code.
    
    Ensure the output is valid, clean HTML with:
    - appropriate section headers
    - formatted paragraphs
    - clickable links
    
    RESPOND only with HTML, without additional commentary.
    
    
    
    '''

    # Call the LLM by interacting with the CLIENT. 
    # Remember to set the correct values for the model, messages (system and user prompts) and temperature
    response = CLIENT.chat.completions.create( 
        # Pass in the model
        model=model,
        messages=[ 
            # System prompt is already defined
            {"role": "system", "content": system_prompt},
            # Add user prompt
            {"role": "user", "content": user_prompt},
        ],
        # Set the temperature equal to the temperature parameter passed to the function
        temperature=0.3
    )

    ### END CODE HERE ###

    # Extract the HTML from the assistant message
    html = response.choices[0].message.content.strip()  

    return html

In [20]:
html = convert_report_to_html(new_report['revised_report'])

In [21]:
print(html)

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Overview of Agentic AI</title>
</head>
<body>
    <h1>Overview of Agentic AI</h1>
    <p>Agentic AI refers to artificial intelligence systems designed to exhibit agency, meaning they can perceive their environment and take autonomous actions to achieve specific goals. This concept extends beyond traditional AI, which often operates under fixed rules and lacks the capability to independently devise strategies or make decisions that reflect self-initiative or adaptive behavior.</p>
    
    <h2>Key Characteristics:</h2>
    <ol>
        <li><strong>Autonomy</strong>: Agentic AI systems can operate independently without human intervention, making decisions based on their programming and dynamic inputs from their environment.</li>
        <li><strong>Goal-Directed Behavior</strong>: These systems possess explicit objectives and uti

In [22]:
display(HTML(html))